# Example 1: Simple `MeanRisk` optimisation

Here we show a simple example of how to use `PortfolioOptimisers`. We will perform the classic Markowitz optimisation.

In [1]:
using PortfolioOptimisers

PrettyTables is used to format the example output.

In [2]:
using PrettyTables

# Format for pretty tables.
tsfmt = (v, i, j) -> begin
    if j == 1
        return Date(v)
    else
        return v
    end
end;
resfmt = (v, i, j) -> begin
    if j == 1
        return v
    else
        return isa(v, Number) ? "$(round(v*100, digits=3)) %" : v
    end
end;
mipresfmt = (v, i, j) -> begin
    if j ∈ (1, 2, 3)
        return v
    else
        return isa(v, Number) ? "$(round(v*100, digits=3)) %" : v
    end
end;

## 1. Load the data

Import the S&P500 data from a compressed `.csv` file. We will only use the last 253 observations.

In [3]:
using CSV, TimeSeries, DataFrames

X = TimeArray(CSV.File(joinpath(@__DIR__, "SP500.csv.gz")); timestamp = :Date)[(end - 252):end]
pretty_table(X[(end - 5):end]; formatters = [tsfmt])

┌────────────┬─────────┬─────────┬─────────┬─────────┬─────────┬─────────┬──────
│  timestamp │    AAPL │     AMD │     BAC │     BBY │     CVX │      GE │     ⋯
│ Dates.Date │ Float64 │ Float64 │ Float64 │ Float64 │ Float64 │ Float64 │ Flo ⋯
├────────────┼─────────┼─────────┼─────────┼─────────┼─────────┼─────────┼──────
│ 2022-12-20 │ 131.916 │   65.05 │  31.729 │  77.371 │ 169.497 │  62.604 │ 310 ⋯
│ 2022-12-21 │ 135.057 │   67.68 │  32.212 │  78.729 │  171.49 │   64.67 │ 314 ⋯
│ 2022-12-22 │ 131.846 │   63.86 │  31.927 │  78.563 │ 168.918 │  63.727 │ 311 ⋯
│ 2022-12-23 │ 131.477 │   64.52 │  32.005 │  79.432 │  174.14 │  63.742 │ 314 ⋯
│ 2022-12-27 │ 129.652 │   63.27 │  32.065 │   79.93 │ 176.329 │  64.561 │ 314 ⋯
│ 2022-12-28 │ 125.674 │   62.57 │  32.301 │  78.279 │ 173.728 │  63.883 │  31 ⋯
└────────────┴─────────┴─────────┴─────────┴─────────┴─────────┴─────────┴──────
                                                              14 columns omitted


First we must compute the returns from the prices. The `ReturnsResult` struct stores the asset names in `nx`, asset returns in `X`, and timestamps in `ts`. The other fields are used in other applications which we will not be showcasing here.

In [4]:
rd = prices_to_returns(X)

ReturnsResult
    nx ┼ 20-element Vector{String}
     X ┼ 252×20 Matrix{Float64}
    nf ┼ nothing
     F ┼ nothing
    ts ┼ 252-element Vector{Dates.Date}
    iv ┼ nothing
  ivpa ┴ nothing


## 2. MeanRisk optimisation

### 2.1 Creating a solver instance

All optimisations require some prior statistics to be computed. This can either be done before the optimisation function, or within it. For certain optimisations, precomputing the prior is more efficient, but it makes no difference here so we'll do it within the optimisation.

The `MeanRisk` estimator defines a mean-risk optimisation problem. It is a `JuMPOptimisationEstimator`, which means it requires a `JuMP`-compatible optimiser, which in this case will be `Clarabel`.

In [5]:
using Clarabel

We have to define a `Solver` object, which contains the optimiser we wish to use, an optional name for logging purposes, optional solver settings, and optional kwargs for [`JuMP.assert_is_solved_and_feasible`](https://jump.dev/JuMP.jl/stable/api/JuMP/#assert_is_solved_and_feasible).

Given the vast range of optimisation options and types, it is often useful to try different solver and settings combinations. To this aim, it is also possible to provide a vector of `Solver` objects, which is iterated over until one succeeds or all fail. The classic Markowitz optimisation is rather simple, so we will use a single solver instance.

In [6]:
slv = Solver(; name = :clarabel1, solver = Clarabel.Optimizer,
             settings = Dict("verbose" => false),
             check_sol = (; allow_local = true, allow_almost = true))

Solver
         name ┼ Symbol: :clarabel1
       solver ┼ UnionAll: Clarabel.MOIwrapper.Optimizer
     settings ┼ Dict{String, Bool}: Dict{String, Bool}("verbose" => 0)
    check_sol ┼ @NamedTuple{allow_local::Bool, allow_almost::Bool}: (allow_local = true, allow_almost = true)
  add_bridges ┴ Bool: true


### 2.2 Defining the optimisation estimator

`PortfolioOptimisers` is designed to heavily leverage composition. The first hint of this design ethos in the examples comes in the form of `JuMPOptimiser`, which is the structure defining the optimiser parameters used in all `JuMPOptimisationEstimator`s.

Let's create a `MeanRisk` estimator. As you can see from the output, `JuMPOptimiser` and `MeanRisk` contain myriad properties that we will not showcase in this example.

In [7]:
mr = MeanRisk(; opt = JuMPOptimiser(; slv = slv))

MeanRisk
  opt ┼ JuMPOptimiser
      │       pr ┼ EmpiricalPrior
      │          │        ce ┼ PortfolioOptimisersCovariance
      │          │           │   ce ┼ Covariance
      │          │           │      │    me ┼ SimpleExpectedReturns
      │          │           │      │       │   w ┴ nothing
      │          │           │      │    ce ┼ GeneralCovariance
      │          │           │      │       │   ce ┼ SimpleCovariance: SimpleCovariance(true)
      │          │           │      │       │    w ┴ nothing
      │          │           │      │   alg ┴ Full()
      │          │           │   mp ┼ DenoiseDetoneAlgMatrixProcessing
      │          │           │      │     pdm ┼ Posdef
      │          │           │      │         │      alg ┼ UnionAll: NearestCorrelationMatrix.Newton
      │          │           │      │         │   kwargs ┴ @NamedTuple{}: NamedTuple()
      │          │           │      │      dn ┼ nothing
      │          │           │      │      dt ┼ nothing

### 2.3 Performing the optimisation

The `optimise` function is used to perform all optimisations in `PortfolioOptimisers`. Each method returns an `AbstractResult` object containing the optimisation results, which include a return code, a solution object, and relevant statistics (precomputed or otherwise) used in the optimisation.

The field `retcode` informs us that our optimisation was successful because it contains an `OptimisationSuccess` return code.

In [8]:
res = optimise(mr, rd)

MeanRiskResult
       oe ┼ DataType: DataType
       pa ┼ ProcessedJuMPOptimiserAttributes
          │       pr ┼ LowOrderPrior
          │          │         X ┼ 252×20 Matrix{Float64}
          │          │        mu ┼ 20-element Vector{Float64}
          │          │     sigma ┼ 20×20 Matrix{Float64}
          │          │      chol ┼ nothing
          │          │         w ┼ nothing
          │          │       ens ┼ nothing
          │          │       kld ┼ nothing
          │          │        ow ┼ nothing
          │          │        rr ┼ nothing
          │          │      f_mu ┼ nothing
          │          │   f_sigma ┼ nothing
          │          │       f_w ┴ nothing
          │       wb ┼ WeightBounds
          │          │   lb ┼ 20-element StepRangeLen{Float64, Base.TwicePrecision{Float64}, Base.TwicePrecision{Float64}, Int64}
          │          │   ub ┴ 20-element StepRangeLen{Float64, Base.TwicePrecision{Float64}, Base.TwicePrecision{Float64}, Int64}
          │ 

Let's view the solution results as a pretty table. For convenience, we have ensured all `AbstractResult` have a property called `w`, which directly accesses `sol.w`. The optimisations don't shuffle the asset order, so we can simply view the asset names and weights side by side.

In [9]:
pretty_table(DataFrame(:assets => rd.nx, :weights => res.w); formatters = [resfmt])

┌────────┬──────────┐
│ assets │  weights │
│ String │  Float64 │
├────────┼──────────┤
│   AAPL │    0.0 % │
│    AMD │    0.0 % │
│    BAC │    0.0 % │
│    BBY │    0.0 % │
│    CVX │  7.432 % │
│     GE │  0.806 % │
│     HD │    0.0 % │
│    JNJ │ 36.974 % │
│    JPM │  0.749 % │
│     KO │ 11.161 % │
│    LLY │    0.0 % │
│    MRK │ 17.467 % │
│   MSFT │    0.0 % │
│    PEP │  8.978 % │
│    PFE │    0.0 % │
│      ⋮ │        ⋮ │
└────────┴──────────┘
       5 rows omitted


## 3. Finite allocation

We have the optimal solution, but most people don't have access to effectively unlimited funds. Given the optimised weights, current prices and a finite cash amount, it is possible to perform a finite allocation. We will use a discrete allocation method which uses mixed-integer programming to find the best allocation. We have another finite allocation method which uses a greedy algorithm that can deal with fractional shares, but we will reserve it for a later example.

For the discrete allocation, we need a solver capable of handling mixed-integer programming problems, we will use `HiGHS`.

In [10]:
using HiGHS

mip_slv = Solver(; name = :highs1, solver = HiGHS.Optimizer,
                 settings = Dict("log_to_console" => false),
                 check_sol = (; allow_local = true, allow_almost = true))
da = DiscreteAllocation(; slv = mip_slv)

DiscreteAllocation
  slv ┼ Solver
      │          name ┼ Symbol: :highs1
      │        solver ┼ DataType: DataType
      │      settings ┼ Dict{String, Bool}: Dict{String, Bool}("log_to_console" => 0)
      │     check_sol ┼ @NamedTuple{allow_local::Bool, allow_almost::Bool}: (allow_local = true, allow_almost = true)
      │   add_bridges ┴ Bool: true
   sc ┼ Int64: 1
   so ┼ Int64: 1
   wf ┼ AbsoluteErrorWeightFinaliser()
   fb ┼ GreedyAllocation
      │     unit ┼ Int64: 1
      │     args ┼ Tuple{}: ()
      │   kwargs ┼ @NamedTuple{}: NamedTuple()
      │       fb ┴ nothing


Luckily, we have the optimal weights, the latest prices are the last entry of our original time array `X`, and let's say we have `4206.9` USD to invest.

The function can optionally take extra positional arguments to account for a variety of fees, but we will not use them here.

In [11]:
mip_res = optimise(da, res.w, vec(values(X[end])), 4206.9)

DiscreteAllocationResult
         oe ┼ DataType: DataType
    retcode ┼ OptimisationSuccess
            │   res ┴ nothing
  s_retcode ┼ nothing
  l_retcode ┼ OptimisationSuccess
            │   res ┴ Dict{Any, Any}: Dict{Any, Any}()
     shares ┼ 20-element SubArray{Float64, 1, Matrix{Float64}, Tuple{Base.Slice{Base.OneTo{Int64}}, Int64}, true}
       cost ┼ 20-element SubArray{Float64, 1, Matrix{Float64}, Tuple{Base.Slice{Base.OneTo{Int64}}, Int64}, true}
          w ┼ 20-element SubArray{Float64, 1, Matrix{Float64}, Tuple{Base.Slice{Base.OneTo{Int64}}, Int64}, true}
       cash ┼ Float64: 8.47199999999711
    s_model ┼ nothing
    l_model ┼ A JuMP Model
            │ ├ solver: HiGHS
            │ ├ objective_sense: MIN_SENSE
            │ │ └ objective_function_type: JuMP.AffExpr
            │ ├ num_variables: 21
            │ ├ num_constraints: 42
            │ │ ├ JuMP.AffExpr in MOI.GreaterThan{Float64}: 1
            │ │ ├ Vector{JuMP.AffExpr} in MOI.NormOneCone: 1
            │ 

The result of this optimisation contains different pieces of information to the previous one. The reason various fields are prefixed by `l_`or `s_` is because the discrete allocation method splits the assets into long and short positions, which are recombined in the final result.

Let's see the results in another pretty table.

In [12]:
pretty_table(DataFrame(:assets => rd.nx, :shares => mip_res.shares, :cost => mip_res.cost,
                       :opt_weights => res.w, :mip_weights => mip_res.w);
             formatters = [mipresfmt])

┌────────┬─────────┬─────────┬─────────────┬─────────────┐
│ assets │  shares │    cost │ opt_weights │ mip_weights │
│ String │ Float64 │ Float64 │     Float64 │     Float64 │
├────────┼─────────┼─────────┼─────────────┼─────────────┤
│   AAPL │     0.0 │     0.0 │       0.0 % │       0.0 % │
│    AMD │     0.0 │     0.0 │       0.0 % │       0.0 % │
│    BAC │     0.0 │     0.0 │       0.0 % │       0.0 % │
│    BBY │     0.0 │     0.0 │       0.0 % │       0.0 % │
│    CVX │     2.0 │ 347.456 │     7.432 % │     8.276 % │
│     GE │     0.0 │     0.0 │     0.806 % │       0.0 % │
│     HD │     0.0 │     0.0 │       0.0 % │       0.0 % │
│    JNJ │     9.0 │ 1566.77 │    36.974 % │    37.318 % │
│    JPM │     0.0 │     0.0 │     0.749 % │       0.0 % │
│     KO │     6.0 │ 375.654 │    11.161 % │     8.947 % │
│    LLY │     0.0 │     0.0 │       0.0 % │       0.0 % │
│    MRK │     7.0 │ 767.067 │    17.467 % │     18.27 % │
│   MSFT │     0.0 │     0.0 │       0.0 % │       0.0 %

We can see that the mip weights do not exactly match the optimal ones, but that is because we only have finite resources. Note that the sum of the costs minus the initial cash is equal to the `cash` property of the result. This changes when we introduce fees, which will be shown in a future example.

In [13]:
println("used cash ≈ available cash: $(isapprox(mip_res.cash, 4206.9 - sum(mip_res.cost)))")

used cash ≈ available cash: true


We can also see that the cost of each asset is equal to the number of shares times its price.

In [14]:
println("cost of shares ≈ cost of portfolio: $(all(isapprox.(mip_res.shares .* vec(values(X[end])), mip_res.cost)))")

cost of shares ≈ cost of portfolio: true


---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*